In [1]:
import numpy as np
from pycococreatortools import pycococreatortools
import os
import glob
import cv2
from tqdm import tqdm
import json
from PIL import Image


In [2]:
home_path=os.path.expanduser('~')
dir0_with_images_and_ann = os.path.join(home_path, 'Pobrane/images/More-weapons-1')

In [3]:
def get_segmentation(in_dict):
    segmentation = []
    if not len(in_dict['contours']):
        return segmentation
    in_dict['contours'] = sorted(in_dict['contours'], key=len, reverse=True)
    for contour_pack in [in_dict['contours'][0]]:
        for contour in contour_pack:
            segmentation.append(contour['x'])
            segmentation.append(contour['y'])

    
    return [segmentation]

def get_bbox(in_dict):
    left = in_dict['bbox']['left']
    top = in_dict['bbox']['top']
    right = in_dict['bbox']['right']
    bottom = in_dict['bbox']['bottom']
    width = right-left
    height = bottom-top
    return [left, top, width, height]

In [4]:
w_id = 1
forbidden_phrases=["weapon", 'M39_EMR','LeftHandedWeapons','Remington_R1',  'Beretta_R93', 'Browning_HP', 'SS2_AssaultRifle',
                   'Heckler_Koch_UMP-45','Remington_700_CDL','VKS_Vyhlop','skorpion','mac11' ,'Colt_1851']
cats_ids = {'Glock_17':5,'45_ACP_Smith_And_Wesson ' :2, 'mp5k' : 7, 
           'M1911A':2, 'AKM':1, 'AK47': 1, 'Colt_M4A1 ': 6,'Colt_Python':3, 'AK_74': 1, 
            'thompson':9, 'Scar_L': 8, 'Dan_Wesson_715':3,
           'AR':6, 'AB-10_LP':10,
            'Enfield':4}
image_id = 0
ann_id = 0
coco_output = {'images': [], "annotations": [], 'type': "instances"}
coco_output['categories'] = [{"supercategory": "weapon","id": 1,"name": "AK-74m_synth"},
                            {"supercategory": "weapon","id": 2,"name": "colt_synth"},
                            {"supercategory": "weapon","id": 3,"name": "danwesson_synth"},
                            {"supercategory": "weapon","id": 4,"name": "enfield_synth"},
                            {"supercategory": "weapon","id": 5,"name": "glock_synth"},
                            {"supercategory": "weapon","id": 6,"name": "m4_synth"},
                            {"supercategory": "weapon","id": 7,"name": "mp5_synth"},
                            {"supercategory": "weapon","id": 8,"name": "scar_synth"},
                            {"supercategory": "weapon","id": 9,"name": "thompson_synth"},
                            {"supercategory": "weapon","id": 10,"name": "ab10_synth"}]
tags = set()
break_count = 0
for src_dir in [dir0_with_images_and_ann]:
    
    images_pathes = glob.glob(src_dir+'/*.png')
    for image_path in tqdm(images_pathes):
        image_id += 1
        image = Image.open(image_path)
        parent_dir, image_name  = image_path.split('/')[-2:]
        image_filename_with_parentdir = os.path.join(parent_dir, image_name)
        image_info = pycococreatortools.create_image_info(
        image_id, image_filename_with_parentdir, image.size)
        
        cutted_image_path = '_'.join(image_path.split('_')[:-1])
        annotation_list = glob.glob(cutted_image_path+'_*.json')
        prev_ann_id = ann_id
        break_for = False
        for annot_path in annotation_list:
            with open(annot_path, 'r') as f:
                desc = json.load(f)
                if desc['tagName'] not in tags and desc['tagName'] not in cats_ids:
                    print(desc['tagName'])
                    tags.add(desc['tagName'])
                if desc['tagName'] in forbidden_phrases:
                    break_count+=1
                    if break_count %50==49:
                        print("break_count", break_count)
                    break
                if desc['tagName'] not in cats_ids:
                    continue
                    
                annotation_info = {}
                annotation_info['segmentation'] = get_segmentation(desc)
                if not len(annotation_info['segmentation']):
                    continue
                ann_id += 1
                annotation_info['area'] = desc['area']
                annotation_info['iscrowd'] = 0
                annotation_info['image_id'] = image_id
                annotation_info['bbox'] = get_bbox(desc)
                annotation_info['category_id'] = cats_ids[desc['tagName']]
                annotation_info['id'] =  ann_id
                coco_output['annotations'].append(annotation_info)
        if prev_ann_id != ann_id:
            coco_output["images"].append(image_info)

  0%|          | 1/76389 [00:01<23:01:16,  1.08s/it]

jeep
bus
Car logo
Throtttle
Door window
People with weapons
Brakes
Heckler_Koch_UMP-45


  0%|          | 2/76389 [00:02<23:09:43,  1.09s/it]

carpet
table
laptop


  0%|          | 3/76389 [00:03<22:59:06,  1.08s/it]

...

break_count 36999


 83%|████████▎ | 63221/76389 [19:54:38<4:14:25,  1.16s/it]

break_count 37049


 83%|████████▎ | 63302/76389 [19:56:13<4:07:18,  1.13s/it]


KeyboardInterrupt: 

In [5]:
annotation_file='/home/m/Pobrane/trainval.json'
os.makedirs(os.path.split(annotation_file)[0], exist_ok=True)
with open(annotation_file, 'w') as f:
    json.dump(coco_output, f)